In [2]:

import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import glob


#Dataset downloaded and unzipped on D drive
train_images = sorted(glob.glob("D:/keras_png_slices_data/keras_png_slices_train/*.png"))
train_masks = sorted(glob.glob("D:/keras_png_slices_data/keras_png_slices_seg_train/*.png"))
val_images = sorted(glob.glob("D:/keras_png_slices_data/keras_png_slices_validate/*.png"))
val_masks = sorted(glob.glob("D:/keras_png_slices_data/keras_png_slices_seg_validate/*.png"))
test_images = sorted(glob.glob("D:/keras_png_slices_data/keras_png_slices_test/*.png"))
test_masks = sorted(glob.glob("D:/keras_png_slices_data/keras_png_slices_seg_test/*.png"))

train_data = tf.data.Dataset.from_tensor_slices((train_images, train_masks))
val_data = tf.data.Dataset.from_tensor_slices((val_images, val_masks))
test_data = tf.data.Dataset.from_tensor_slices((test_images, test_masks))
train_data = train_data.shuffle(len(train_images))
val_data = val_data.shuffle(len(val_images))
test_data = test_data.shuffle(len(test_images))


def map_fn(image_fp, mask_fp):
    image = tf.io.read_file(image_fp)
    image = tf.image.decode_png(image, channels=1)
    image = tf.image.resize(image, (256, 256))
    image = tf.cast(image, tf.float32) /255.0
    
    mask = tf.io.read_file(mask_fp)
    mask = tf.image.decode_png(mask, channels=1)
    mask = tf.image.resize(mask, (256, 256))
    mask = mask == [0, 85, 170, 255]
    mask = tf.cast(mask, tf.uint8)
    return image, mask

def display(display_list):
    plt.figure(figsize = (10, 10))
    for i in range(len(display_list)):
        plt.subplot(1, len(display_list), i+1)
        plt.imshow(display_list[i], cmap='gray')
        plt.axis('off')
    plt.show()

train_data = train_data.map(map_fn)
val_data = val_data.map(map_fn)
test_data = test_data.map(map_fn)


#Must reference for later on
def dice_coef(y_true, y_pred):
    y_true_f = tf.keras.backend.flatten(y_true)
    y_pred_f = tf.keras.backend.flatten(y_pred)
    intersection = tf.keras.backend.sum(y_true_f * y_pred_f)
    return (2. * intersection + tf.keras.backend.epsilon()) / (tf.keras.backend.sum(y_true_f) + tf.keras.backend.sum(y_pred_f)
                                                               + tf.keras.backend.epsilon())

#unet model

def unet_model(output_channels, f = 64):
    inputs = tf.keras.layers.Input(shape=(256, 256, 1))
    
    d1 = tf.keras.layers.Conv2D(f, 3, padding='same', activation='relu')(inputs)
    d1 = tf.keras.layers.Conv2D(f, 3, padding='same', activation='relu')(d1)
    
    d2 = tf.keras.layers.MaxPooling2D()(d1)
    d2 = tf.keras.layers.Conv2D(2*f, 3, padding='same', activation='relu')(d2)
    d2 = tf.keras.layers.Conv2D(2*f, 3, padding='same', activation='relu')(d2)
    
    d3 = tf.keras.layers.MaxPooling2D()(d2)
    d3 = tf.keras.layers.Conv2D(4*f, 3, padding='same', activation='relu')(d3)
    d3 = tf.keras.layers.Conv2D(4*f, 3, padding='same', activation='relu')(d3)
    
    d4 = tf.keras.layers.MaxPooling2D()(d3)
    d4 = tf.keras.layers.Conv2D(8*f, 3, padding='same', activation='relu')(d4)
    d4 = tf.keras.layers.Conv2D(8*f, 3, padding='same', activation='relu')(d4)
    
    d5 = tf.keras.layers.MaxPooling2D()(d4)
    d5 = tf.keras.layers.Conv2D(16*f, 3, padding='same', activation='relu')(d5)
    d5 = tf.keras.layers.Conv2D(16*f, 3, padding='same', activation='relu')(d5)
    
    u4 = tf.keras.layers.UpSampling2D()(d5)
    u4 = tf.keras.layers.concatenate([u4, d4])
    u4 = tf.keras.layers.Conv2D(8*f, 3, padding='same', activation='relu')(u4)
    u4 = tf.keras.layers.Conv2D(8*f, 3, padding='same', activation='relu')(u4)
    
    u3 = tf.keras.layers.UpSampling2D()(u4) 
    u3 = tf.keras.layers.concatenate([u3, d3])
    u3 = tf.keras.layers.Conv2D(4*f, 3, padding ='same', activation='relu')(u3)
    u3 = tf.keras.layers.Conv2D(4*f, 3, padding ='same', activation='relu')(u3)
    
    u2 = tf.keras.layers.UpSampling2D()(u3) 
    u2 = tf.keras.layers.concatenate([u2, d2])
    u2 = tf.keras.layers.Conv2D(2*f, 3, padding ='same', activation='relu')(u2)
    u2 = tf.keras.layers.Conv2D(2*f, 3, padding ='same', activation='relu')(u2)
    
    u1 = tf.keras.layers.UpSampling2D()(u2) 
    u1 = tf.keras.layers.concatenate([u1, d1])
    u1 = tf.keras.layers.Conv2D(f, 3, padding ='same', activation='relu')(u1)
    u1 = tf.keras.layers.Conv2D(f, 3, padding ='same', activation='relu')(u1)
    
    #last layer
    outputs = tf.keras.layers.Conv2D(output_channels, 1, activation='softmax')(u1)
    
    return tf.keras.Model(inputs=inputs, outputs=outputs)

model = unet_model(4, f=4)
model.compile(optimizer='adam',
             loss ='categorical_crossentropy',
             metrics=[dice_coef, 'accuracy'])

def show_predictions(ds, num=1):
    for image, mask in ds.take(num):
        pred_mask = model.predict(image[tf.newaxis, ...])
        pred_mask = tf.argmax(pred_mask[0], axis=-1)
        display([tf.squeeze(image), tf.argmax(mask, axis=-1), pred_mask])
    
show_predictions(val_data)
    


C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passi

ImportError: Keras requires TensorFlow 2.2 or higher. Install TensorFlow via `pip install tensorflow`

In [1]:
history = model.fit(train_data.batch(32), epochs = 3,
                   validation_data = val_data.batch(32))

NameError: name 'model' is not defined

In [ ]:
show_predictions(val_data)
